<a href="https://colab.research.google.com/github/fabio-weydson/sandbox-datascience/blob/main/CE_5_2_por_GPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo de caso 5.2: Rastreando a localização de um objeto que se move a uma velocidade constante

**Configuração do *notebook***

In [ ]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

## Geração de dados

Precisamos gerar sinteticamente medições ruidosas da velocidade constante em ambas as direções. Em Python, podemos fazer isso da seguinte maneira:

In [ ]:
# Número de medições
m = 100
# velocidade em x (constante)
vx= 10
# velocidade em y (constante)
vy= 10
# adicionamos ruído aleatório às duas medições
mx = np.array(vx + np.random.randn(m)) 
my = np.array(vy + np.random.randn(m)) 
measurements = np.vstack((mx,my))

Visualização dos dados gerados:

In [ ]:
plt.plot(range(m),mx, label='$v_1 (medições)$') 
plt.plot(range(m),my, label='$v_2 (medições)$') 
plt.ylabel('Medições de velocidade')
plt.title('Medições com ruído') 
plt.legend(loc='best',prop={'size':12})

plt.show()

## Inicialização de variáveis

Podemos inicializar as variáveis ​​e matrizes que são especificadas no modelo da seguinte maneira:

In [ ]:
# Tamanho do intervalo para o filtro
dt = 0.1
# Matriz de identidade
I = np.eye(4)
# Matriz de estado
x = np.matrix([[0.0, 0.0, 0.0, 0.0]]).T
# Matriz P
P = np.diag([1000.0, 1000.0, 1000.0, 1000.0])
# Matriz A
A = np.matrix([[1.0, 0.0, dt, 0.0], 
               [0.0, 1.0, 0.0, dt],
               [0.0, 0.0, 1.0, 0.0],
               [0.0, 0.0, 0.0, 1.0]])
# Matriz H
H = np.matrix([[0.0, 0.0, 1.0, 0.0], 
               [0.0, 0.0, 0.0, 1.0]])
# Matriz R
r = 100.0
R = np.matrix([[r, 0.0],
               [0.0, r]])
# Matrizes Q e G
s = 8.8
G = np.matrix([[0.5*dt**2],
               [0.5*dt**2],
               [dt],
               [dt]])
Q = G*G.T*s**2

## Algoritmo do Filtro de Kalman

In [ ]:
# As seguintes variáveis irão salvar os resultados em cada iteração
xt = [] 
yt = [] 
dxt= [] 
dyt= [] 
Zx = [] 
Zy = [] 
Px = [] 
Py = [] 
Pdx= [] 
Pdy= [] 
Rdx= [] 
Rdy= [] 
Kx = [] 
Ky = [] 
Kdx= [] 
Kdy= []

**Algoritmo do Filtro de Kalman**

In [ ]:
for n in range(len(measurements[0])):
  # Previsão
  # Previsão de estado
  x = A*x
  # Previsão da covariância do erro
  P = A*P*A.T + Q
  # Atualizar intervalos
  # Ganho do Kalman
  S = H*P*H.T + R
  K = (P*H.T) * np.linalg.pinv(S)
  # Atualizar a estimativa via Z
  Z = measurements[:,n].reshape(2,1) 
  y = Z - (H*x)
  x = x + (K*y)
  # Covariância do erro
  P = (I - (K*H))*P
  
  # Salvando resultados
  xt.append(float(x[0])) 
  yt.append(float(x[1])) 
  dxt.append(float(x[2])) 
  dyt.append(float(x[3])) 
  Zx.append(float(Z[0])) 
  Zy.append(float(Z[1])) 
  Px.append(float(P[0,0])) 
  Py.append(float(P[1,1])) 
  Pdx.append(float(P[2,2])) 
  Pdy.append(float(P[3,3])) 
  Rdx.append(float(R[0,0])) 
  Rdy.append(float(R[1,1])) 
  Kx.append(float(K[0,0])) 
  Ky.append(float(K[1,0])) 
  Kdx.append(float(K[2,0])) 
  Kdy.append(float(K[3,0]))

## Resultados

Medição da velocidade:

In [ ]:
# Nossas estimativas em vermelho
plt.plot(range(len(measurements[0])),dxt, label='$v_1$', c='r') 
plt.plot(range(len(measurements[0])),dyt, label='$v_2$', c='r')

# As medições ruidosas da velocidade em ambas as direções estão em verde e azul 
plt.plot(range(len(measurements[0])),mx, label='$z_1 (medições)$', c='g') 
plt.plot(range(len(measurements[0])),my, label='$z_2 (medições)$', c='b')

# Velocidade constante nas duas direções na cor preta
plt.axhline(vx, color='#999999', label='$v_1(real)$') 
plt.axhline(vy, color='#999999', label='$v_2(real)$')
plt.title('Estimativas da velocidade') 
plt.legend(loc='best') 
plt.ylim([0, 20])
plt.show()

Rastreamento da posição:

In [ ]:
# Scatter plot da estimativa da posição de x e y na cor preta
# idealmente deveiam formar uma linha reta
plt.scatter(xt,yt, s=20, label='State', c='black')

# ponto inicial em verde e ponto final em vermelho
plt.scatter(xt[0],yt[0], s=100, label='Início', c='g') 
plt.scatter(xt[-1],yt[-1], s=100, label='Objetivo', c='r')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.title('Estimativas da posição (tracking)') 
plt.legend(loc='best')
plt.show()

---

Referências:

* https://balzer82.github.io/Kalman/